In [43]:
import requests
from bs4 import BeautifulSoup
import bs4
import html
import re

In [ ]:
main_url = "https://devgan.in/ipc"

In [ ]:
page = requests.get(main_url)

In [ ]:
page.text

In [ ]:
page.content

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
menu_links = soup.select('.menu tr td a')

In [ ]:
links_list = [link.get('href')[5:] for link in menu_links]

In [ ]:
links_list

In [ ]:
link_urls = [(main_url+"/"+link, link[:-4]) for link in links_list]

In [ ]:
link_urls

In [ ]:
base_url = "https://devgan.in/ipc"

### Extracting text from content html

In [ ]:
page1 = requests.get('https://devgan.in/ipc/chapter_01.php')

In [49]:
def get_text(tag: bs4.Tag) -> str:
    _inline_elements = {"a","span","em","strong","u","i","font","mark","label",
    "s","sub","sup","tt","bdo","button","cite","del","b","a","font",}

    for element in tag(text=lambda text: isinstance(text, bs4.Comment)):
      element.extract()

    def _get_text(tag: bs4.Tag):
        for child in tag.children:

            if isinstance(child, bs4.Tag):
                # if the tag is a block type tag then yield new lines before after
                is_block_element = child.name not in _inline_elements
                if is_block_element:
                    yield "\n"
                yield from ["\n"] if child.name == "br" else _get_text(child)
                if is_block_element:
                    yield "\n"
            elif isinstance(child, bs4.NavigableString):
                yield child.string

    return re.sub(r'\n\s*\n', '\n', "".join(_get_text(tag)))

In [ ]:
soup1 = BeautifulSoup(page1.text, 'html.parser')
content1 = soup1.find('div', id='content')
print(get_text(content1))

## final conversion


In [46]:
from concurrent.futures import ThreadPoolExecutor
import os

In [ ]:
os.makedirs('pages', exist_ok=True)

In [ ]:
def download(url):
  pg = requests.get(url[0])
  with open(f'pages/{url[1]}.txt', 'w') as f:
    f.write(pg.text)

In [ ]:
# download html files
with ThreadPoolExecutor(max_workers=16) as executor:
    executor.map(download, link_urls)  # urls=[list of url]


In [51]:
# extract data from html
for _, name in link_urls:
  with open(f'pages/{name}.txt', 'r') as f:
    pgdata = f.read()
    sp = BeautifulSoup(pgdata, 'html.parser')
    c1 = sp.find('div', id='content')
    out = get_text(c1)
    with open('raw_main.txt','a') as f1:
      f1.write(out)